In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from pathlib import Path

import os
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

IMG_SIZE = (224, 224)
weights_path = "results/images/baseline_image_weights_only.h5"

# Rebuild same architecture as training
base = tf.keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=IMG_SIZE + (3,),
    weights="imagenet",
    pooling="avg"
)
base.trainable = False

model = tf.keras.Sequential([
    base,
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")
])

# Load learned weights from training
model.load_weights(weights_path)
print("✅ Loaded trained weights successfully.")


✅ Loaded trained weights successfully.


In [5]:
# Extract embeddings from the EfficientNet base (before Dense layer)
# --- Correct way to get EfficientNet feature extractor from a Sequential model ---
# model.layers[0]  → EfficientNetB0 base
base = model.layers[0]

# Rebuild a clean functional model from EfficientNet's own input/output
feature_extractor = tf.keras.Model(
    inputs=base.input,
    outputs=base.output
)

print("✅ Feature extractor ready. Output shape:", feature_extractor.output.shape)



✅ Feature extractor ready. Output shape: (None, 1280)


In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm

IMG_SIZE = (224, 224)
df = pd.read_csv("data/processed/images/rep_index.csv")

def decode_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img

batch_size = 16
paths = df["rep_slice"].astype(str).tolist()
ids = df["series_id"].tolist()

embeddings = []
for i in tqdm(range(0, len(paths), batch_size)):
    batch_paths = paths[i:i+batch_size]
    imgs = tf.stack([decode_img(p) for p in batch_paths])
    feats = feature_extractor.predict(imgs, verbose=0)
    embeddings.append(feats)

embeddings = np.vstack(embeddings)
print("✅ Embedding shape:", embeddings.shape)


100%|██████████| 2/2 [00:03<00:00,  1.61s/it]

✅ Embedding shape: (23, 1280)


In [8]:
from pathlib import Path
Path("data/processed/images").mkdir(parents=True, exist_ok=True)

embed_df = pd.DataFrame(embeddings)
embed_df.insert(0, "series_id", ids)
embed_df.to_csv("data/processed/images/image_embeddings_1280.csv", index=False)

print("✅ Saved 1280-D embeddings to data/processed/images/image_embeddings_1280.csv")


✅ Saved 1280-D embeddings to data/processed/images/image_embeddings_1280.csv


In [10]:
import tensorflow as tf
save_dir = "results/images/feature_extractor_keras_tf"
tf.saved_model.save(feature_extractor, save_dir)
print("✅ Feature extractor saved safely using tf.saved_model.save.")


INFO:tensorflow:Assets written to: results/images/feature_extractor_keras_tf\assets


INFO:tensorflow:Assets written to: results/images/feature_extractor_keras_tf\assets


✅ Feature extractor saved safely using tf.saved_model.save.
